In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime as dt

LEAGUE_ = 'CH'
if LEAGUE_ == 'PL':
    TEAM_NUM = 20
else:
    TEAM_NUM = 24    

In [ ]:
if LEAGUE_ == 'PL':
    loc="../data/datasets/premier_league/"
else:
    loc="../data/datasets/championship/"    

raw_data_13 = pd.read_csv(loc + "2022-23.csv")
raw_data_12 = pd.read_csv(loc + "2021-22.csv")
raw_data_11 = pd.read_csv(loc + "2020-21.csv")
raw_data_10 = pd.read_csv(loc + "2019-20.csv")
raw_data_9 = pd.read_csv(loc + "2018-19.csv")

raw_data_8=pd.read_csv(loc+"2017-18.csv")
raw_data_7=pd.read_csv(loc+"2016-17.csv")
raw_data_6=pd.read_csv(loc+"2015-16.csv")
raw_data_5=pd.read_csv(loc+"2014-15.csv")
if LEAGUE_ == 'CH':
    raw_data_5 = raw_data_5.copy().iloc[0:TEAM_NUM*(TEAM_NUM-1),:]
raw_data_4=pd.read_csv(loc+"2013-14.csv")
raw_data_3=pd.read_csv(loc+"2012-13.csv")
raw_data_2=pd.read_csv(loc+"2011-12.csv")
raw_data_1=pd.read_csv(loc+"2010-11.csv")



In [ ]:
columns_req = ['Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR','HS','AS','HR','AR','B365H','B365D','B365A']
playing_statistics_1=raw_data_1[columns_req]
playing_statistics_2=raw_data_2[columns_req]
playing_statistics_3=raw_data_3[columns_req]
playing_statistics_4=raw_data_4[columns_req]
playing_statistics_5=raw_data_5[columns_req]
playing_statistics_6=raw_data_6[columns_req]
playing_statistics_7=raw_data_7[columns_req]
playing_statistics_8=raw_data_8[columns_req]

playing_statistics_9=raw_data_9[columns_req]
playing_statistics_10=raw_data_10[columns_req]
playing_statistics_11=raw_data_11[columns_req]
playing_statistics_12=raw_data_12[columns_req]
playing_statistics_13=raw_data_13[columns_req]



In [ ]:
playing_statistics_2.head()

In [ ]:
def get_matchres(playing_stat):
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean('HR').T.columns:
        teams[i] = []

    # the value corresponding to keys is a list containing the match result
    for i in range(len(playing_stat)):
        if playing_stat.iloc[i].FTR == 'H':
            teams[playing_stat.iloc[i].HomeTeam].append('W')
            teams[playing_stat.iloc[i].AwayTeam].append('L')
        elif playing_stat.iloc[i].FTR == 'A':
            teams[playing_stat.iloc[i].AwayTeam].append('W')
            teams[playing_stat.iloc[i].HomeTeam].append('L')
        else:
            teams[playing_stat.iloc[i].AwayTeam].append('D')
            teams[playing_stat.iloc[i].HomeTeam].append('D')
    
    return pd.DataFrame(data=teams, index = [i for i in range(1,(TEAM_NUM * 2 - 1))]).T

In [ ]:
def get_form(playing_stat,num):
    form = get_matchres(playing_stat)
    form_final = form.copy()
    for i in range(num,(TEAM_NUM * 2 - 1)):
        form_final[i] = ''
        j = 0
        while j < num:
            form_final[i] += form[i-j]
            j += 1           
    return form_final

def add_form(playing_stat,num):
    form = get_form(playing_stat,num)
    h = ['D' for i in range(num * int(TEAM_NUM / 2))]  # since form is not available for n MW (n*10)
    a = ['D' for i in range(num * int(TEAM_NUM / 2))]
    
    j = num
    for i in range((num*int(TEAM_NUM / 2)), TEAM_NUM * (TEAM_NUM - 1)):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        
        past = form.loc[ht][j]               # get past n results
        h.append(past[num-1])                    # 0 index is most recent
        
        past = form.loc[at][j]               # get past n results.
        a.append(past[num-1])                   # 0 index is most recent
        
        if ((i + 1)% int((TEAM_NUM / 2))) == 0:
            j = j + 1
    
    playing_stat_copy = playing_stat.copy()
    playing_stat_copy.loc[:,'HM' + str(num)] = h[:playing_stat.shape[0]]                 
    playing_stat_copy.loc[:,'AM' + str(num)] = a[:playing_stat.shape[0]]

    
    return playing_stat_copy


def add_form_df(playing_statistics):
    playing_statistics = add_form(playing_statistics,1)
    playing_statistics = add_form(playing_statistics,2)
    playing_statistics = add_form(playing_statistics,3)
    playing_statistics = add_form(playing_statistics,4)
    playing_statistics = add_form(playing_statistics,5)
    # playing_statistics = add_form(playing_statistics,6)
    return playing_statistics    
    
# Make changes to df
playing_statistics_1 = add_form_df(playing_statistics_1)
playing_statistics_2 = add_form_df(playing_statistics_2)
playing_statistics_3 = add_form_df(playing_statistics_3)
playing_statistics_4 = add_form_df(playing_statistics_4)
playing_statistics_5 = add_form_df(playing_statistics_5)
playing_statistics_6 = add_form_df(playing_statistics_6)
playing_statistics_7 = add_form_df(playing_statistics_7)
playing_statistics_8 = add_form_df(playing_statistics_8)

playing_statistics_9 = add_form_df(playing_statistics_9)
playing_statistics_10 = add_form_df(playing_statistics_10)
playing_statistics_11 = add_form_df(playing_statistics_11)
playing_statistics_12 = add_form_df(playing_statistics_12)
playing_statistics_13 = add_form_df(playing_statistics_13)



In [ ]:
playing_statistics_2.head()

In [ ]:
import sqlite3

# 1.硬盘上创建连接 
con = sqlite3.connect(f"file:../data/db/soccer.db?mode=rw", uri=True) 
# 获取cursor对象 
cur = con.cursor() 
sql = 'insert into Teams(id, name) values(?, ?)' 

# 查询队伍ID
def getTeamID(name):
    # 如果队伍已存在，则跳过
    query = 'select id from Teams where name = ?'
    cur.execute(query, (name,))
    teamId = cur.fetchone()[0]
    return float(teamId)

getTeamID('Arsenal')

# winner字段转换
def getWinner(ftr):
    if ftr == 'H':
        return 1.0
    elif ftr == 'A':
        return 2.0
    else:
        return 0.0
    
def get_points(result):
    if result == 'W':
        return 3.0
    elif result == 'D':
        return 1.0
    else:
        return 0.0  
    
def getFixturesCount():
    con = sqlite3.connect(f"file:../data/db/soccer.db?mode=rw", uri=True) 
    # 获取cursor对象 
    cur = con.cursor()    

    query = 'select count(*) from Fixtures'
    try: 
        cur.execute(query)
        count = cur.fetchone()[0]
        return count
    except Exception as e: 
        print(e) 
        print('创建表失败') 
    finally: 
        # 关闭游标 
        cur.close() 
        # 关闭连接 
        con.close() 

In [ ]:
import uuid

def createData(playing_statistics, season, fixtures, teamStats, round):
    # fixtures数据
    ID_F = []
    DATE = []
    SEASON = []
    LEAGUE = []
    HOME_TEAM_ID = []
    AWAY_TEAM_ID = []
    HOME_GOALS = []
    AWAY_GOALS = []
    WINNER = []
    ODDSDC_1X = []
    ODDSDC_X2 = []
    HOMESTATSID = []
    AWAYSTATSID = []

    # teamStats数据
    ID_TS = []
    FIXTRUEID = []
    TEAMID = []
    RATTING = []
    ERRORS = []
    GOALS = []
    RED_CARDS = []
    SHOTS = []
    OPPONENT_GOALS = []
    OPPONENT_SHOTS = []
    OPPONENT_RATING = []

    for i in range(TEAM_NUM*(TEAM_NUM-1)):
        fixture_id = float(i+1 + round * TEAM_NUM*(TEAM_NUM-1) + getFixturesCount())
        home_stats_id = str(uuid.uuid4())
        away_stats_id = str(uuid.uuid4())

        if len(playing_statistics.iloc[i].Date) == 8:
            dateTemp = dt.strptime(playing_statistics.iloc[i].Date, "%d/%m/%y")
        elif len(playing_statistics.iloc[i].Date) == 10:
            dateTemp = dt.strptime(playing_statistics.iloc[i].Date, "%d/%m/%Y")
        date = dateTemp.strftime("%Y%m%d")
        
        home_team_id = getTeamID(playing_statistics.iloc[i].HomeTeam)
        away_team_id = getTeamID(playing_statistics.iloc[i].AwayTeam)
        home_goals = float(playing_statistics.iloc[i].FTHG)
        away_goals = float(playing_statistics.iloc[i].FTAG)
        winner = getWinner(playing_statistics.iloc[i].FTR)
        oddx_1x = 1/(1/playing_statistics.iloc[i].B365H + 1/playing_statistics.iloc[i].B365D)
        oddx_x2 = 1/(1/playing_statistics.iloc[i].B365D + 1/playing_statistics.iloc[i].B365A)

        # fixtures
        ID_F.append(fixture_id)
        DATE.append(date)
        SEASON.append(season)
        LEAGUE.append(LEAGUE_)
        HOME_TEAM_ID.append(home_team_id)
        AWAY_TEAM_ID.append(away_team_id)
        HOME_GOALS.append(home_goals)
        AWAY_GOALS.append(away_goals)
        WINNER.append(winner)
        ODDSDC_1X.append(oddx_1x)
        ODDSDC_X2.append(oddx_x2)
        HOMESTATSID.append(home_stats_id)
        AWAYSTATSID.append(away_stats_id)

        # home team stats
        if pd.isna(playing_statistics.iloc[i].HR) or pd.isnull(playing_statistics.iloc[i].HR):
            home_red_cards = float(0)
        else:
            home_red_cards = float(playing_statistics.iloc[i].HR)

        if pd.isnull(playing_statistics.iloc[i].HS) or pd.isna(playing_statistics.iloc[i].HS):
            home_shots = 0
        else:
            home_shots = float(playing_statistics.iloc[i].HS)


        # away team
        if pd.isna(playing_statistics.iloc[i].AR) or pd.isnull(playing_statistics.iloc[i].AR):
            away_red_cards = float(0)
        else:
            away_red_cards = float(playing_statistics.iloc[i].AR)

        if pd.isna(playing_statistics.iloc[i].AS) or pd.isnull(playing_statistics.iloc[i].AS):
            away_shots = float(0)
        else:
            away_shots = float(playing_statistics.iloc[i].AS)    
            
        # home rating    
        home_rating = get_points(playing_statistics.iloc[i].HM1)
        home_rating += get_points(playing_statistics.iloc[i].HM2)
        home_rating += get_points(playing_statistics.iloc[i].HM3)
        home_rating += get_points(playing_statistics.iloc[i].HM4)
        home_rating += get_points(playing_statistics.iloc[i].HM5)
        # home_rating += get_points(playing_statistics.iloc[i].HM6)

        # away rating 
        away_rating = get_points(playing_statistics.iloc[i].AM1)
        away_rating += get_points(playing_statistics.iloc[i].AM2)
        away_rating += get_points(playing_statistics.iloc[i].AM3)
        away_rating += get_points(playing_statistics.iloc[i].AM4)
        away_rating += get_points(playing_statistics.iloc[i].AM5)
        # away_rating += get_points(playing_statistics.iloc[i].AM6)

        # home team stats
        ID_TS.append(home_stats_id)
        FIXTRUEID.append(fixture_id)
        TEAMID.append(home_team_id)
        RATTING.append(home_rating)
        ERRORS.append(0.0)
        GOALS.append(home_goals)
        RED_CARDS.append(home_red_cards)
        SHOTS.append(home_shots)
        OPPONENT_GOALS.append(away_goals)
        OPPONENT_SHOTS.append(away_shots)
        OPPONENT_RATING.append(away_rating)
        
        # away team stats
        ID_TS.append(away_stats_id)
        FIXTRUEID.append(fixture_id)
        TEAMID.append(away_team_id)
        RATTING.append(away_rating)
        ERRORS.append(0.0)
        GOALS.append(away_goals)
        RED_CARDS.append(away_red_cards)
        SHOTS.append(away_shots)
        OPPONENT_GOALS.append(home_goals)
        OPPONENT_SHOTS.append(home_shots)
        OPPONENT_RATING.append(home_rating)

    fixtures['id'] = ID_F
    fixtures['date'] = DATE
    fixtures['season'] = SEASON
    fixtures['league'] = LEAGUE
    fixtures['homeTeamID'] = HOME_TEAM_ID
    fixtures['awayTeamID'] = AWAY_TEAM_ID
    fixtures['home_goals'] = HOME_GOALS
    fixtures['away_goals'] = AWAY_GOALS
    fixtures['winner'] = WINNER
    fixtures['oddsDC_1X'] = ODDSDC_1X
    fixtures['oddsDC_X2'] = ODDSDC_X2
    fixtures['homeStatsID'] = HOMESTATSID
    fixtures['awayStatsID'] = AWAYSTATSID

    teamStats['id'] = ID_TS
    teamStats['fixtureID'] = FIXTRUEID
    teamStats['teamID'] = TEAMID
    teamStats['rating'] = RATTING
    teamStats['errors'] = ERRORS
    teamStats['goals'] = GOALS
    teamStats['red_cards'] = RED_CARDS
    teamStats['shots'] = SHOTS
    teamStats['opponent_goals'] = OPPONENT_GOALS
    teamStats['opponent_shots'] = OPPONENT_SHOTS
    teamStats['opponent_rating'] = OPPONENT_RATING

    return fixtures, teamStats

In [ ]:
fixtures_1, teamStats_1 = createData(playing_statistics_1, '2010', pd.DataFrame(data=None, columns=[]), pd.DataFrame(data=None, columns=[]), 0)
fixtures_2, teamStats_2 = createData(playing_statistics_2, '2011', pd.DataFrame(data=None, columns=[]), pd.DataFrame(data=None, columns=[]), 1)
fixtures_3, teamStats_3 = createData(playing_statistics_3, '2012', pd.DataFrame(data=None, columns=[]), pd.DataFrame(data=None, columns=[]), 2)
fixtures_4, teamStats_4 = createData(playing_statistics_4, '2013', pd.DataFrame(data=None, columns=[]), pd.DataFrame(data=None, columns=[]), 3)
fixtures_5, teamStats_5 = createData(playing_statistics_5, '2014', pd.DataFrame(data=None, columns=[]), pd.DataFrame(data=None, columns=[]), 4)
fixtures_6, teamStats_6 = createData(playing_statistics_6, '2015', pd.DataFrame(data=None, columns=[]), pd.DataFrame(data=None, columns=[]), 5)
fixtures_7, teamStats_7 = createData(playing_statistics_7, '2016', pd.DataFrame(data=None, columns=[]), pd.DataFrame(data=None, columns=[]), 6)
fixtures_8, teamStats_8 = createData(playing_statistics_8, '2017', pd.DataFrame(data=None, columns=[]), pd.DataFrame(data=None, columns=[]), 7)

fixtures_9, teamStats_9 = createData(playing_statistics_9, '2018', pd.DataFrame(data=None, columns=[]), pd.DataFrame(data=None, columns=[]), 8)
fixtures_10, teamStats_10 = createData(playing_statistics_10, '2019', pd.DataFrame(data=None, columns=[]), pd.DataFrame(data=None, columns=[]), 9)
fixtures_11, teamStats_11 = createData(playing_statistics_11, '2020', pd.DataFrame(data=None, columns=[]), pd.DataFrame(data=None, columns=[]), 10)
fixtures_12, teamStats_12 = createData(playing_statistics_12, '2021', pd.DataFrame(data=None, columns=[]), pd.DataFrame(data=None, columns=[]), 11)
fixtures_13, teamStats_13 = createData(playing_statistics_13, '2022', pd.DataFrame(data=None, columns=[]), pd.DataFrame(data=None, columns=[]), 12)


In [ ]:
fixtures_2.iloc[0]

In [ ]:
#导入sqllite3模块 
import sqlite3

# 1.硬盘上创建连接 
con = sqlite3.connect(f"file:../data/db/soccer.db?mode=rw", uri=True) 
# 获取cursor对象 
cur = con.cursor() 
sql = '''insert into Fixtures(id, date, season, league, homeTeamID, awayTeamID, home_goals,
            away_goals, winner, oddsDC_1X, oddsDC_X2, homeStatsID, awayStatsID) 
            values(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'''
    
def insertFixtureData(fixtures):
    try: 
        for i in range(TEAM_NUM*(TEAM_NUM-1)):
            cur.execute(sql, (fixtures.iloc[i].id, fixtures.iloc[i].date, fixtures.iloc[i].season,
                              fixtures.iloc[i].league, fixtures.iloc[i].homeTeamID, fixtures.iloc[i].awayTeamID,
                              fixtures.iloc[i].home_goals, fixtures.iloc[i].away_goals, fixtures.iloc[i].winner, 
                              fixtures.iloc[i].oddsDC_1X, fixtures.iloc[i].oddsDC_X2, fixtures.iloc[i].homeStatsID,
                              fixtures.iloc[i].awayStatsID))
            #提交事务 
            con.commit() 
            print('插入成功')
    except Exception as e: 
        print(e) 
        print('插入失败') 
        con.rollback() 
    finally: 
        return


insertFixtureData(fixtures_1)
insertFixtureData(fixtures_2)
insertFixtureData(fixtures_3)
insertFixtureData(fixtures_4)
insertFixtureData(fixtures_5)
insertFixtureData(fixtures_6)
insertFixtureData(fixtures_7)
insertFixtureData(fixtures_8)

insertFixtureData(fixtures_9)
insertFixtureData(fixtures_10)
insertFixtureData(fixtures_11)
insertFixtureData(fixtures_12)
insertFixtureData(fixtures_13)


# 关闭游标 
cur.close() 
# 关闭连接 
con.close()

In [ ]:
#导入sqllite3模块 
import sqlite3

# 1.硬盘上创建连接 
con = sqlite3.connect(f"file:../data/db/soccer.db?mode=rw", uri=True) 
# 获取cursor对象 
cur = con.cursor() 
sql = '''insert into TeamStats(id, fixtureID, teamID, rating, errors, goals, red_cards, shots,
    opponent_goals, opponent_shots, opponent_rating) values(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'''
    
def insertFixtureData(teamStats):
    try: 
        for i in range(TEAM_NUM*(TEAM_NUM-1)*2):
            cur.execute(sql, (teamStats.iloc[i].id, teamStats.iloc[i].fixtureID, teamStats.iloc[i].teamID,
                              teamStats.iloc[i].rating, teamStats.iloc[i].errors, teamStats.iloc[i].goals,
                              teamStats.iloc[i].red_cards, teamStats.iloc[i].shots, teamStats.iloc[i].opponent_goals,
                              teamStats.iloc[i].opponent_shots, teamStats.iloc[i].opponent_rating,))
            #提交事务 
            con.commit() 
            print('插入成功')
    except Exception as e: 
        print(e) 
        print('插入失败') 
        con.rollback() 
    finally: 
        return


insertFixtureData(teamStats_1)
insertFixtureData(teamStats_2)
insertFixtureData(teamStats_3)
insertFixtureData(teamStats_4)
insertFixtureData(teamStats_5)
insertFixtureData(teamStats_6)
insertFixtureData(teamStats_7)
insertFixtureData(teamStats_8)

insertFixtureData(teamStats_9)
insertFixtureData(teamStats_10)
insertFixtureData(teamStats_11)
insertFixtureData(teamStats_12)
insertFixtureData(teamStats_13)


# 关闭游标 
cur.close() 
# 关闭连接 
con.close()